In [330]:
import pandas as pd
from matplotlib import pyplot as plt    
# import seaborn as sns   
import numpy as np
import re
from datetime import datetime
from typing import List, Dict, Tuple, Literal, Optional
from dataclasses import dataclass
    
from scipy.stats import norm



In [331]:
%load_ext autoreload
%autoreload 2

from src.ingest_csv import huckleberry_reader

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [332]:
## Reference documents
## https://cdn.who.int/media/docs/default-source/child-growth/growth-reference-5-19-years/computation.pdf?sfvrsn=c2ff6a95_4

In [333]:
# %conda install -c plotly plotly

from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *

In [397]:
class Zind:
        #           [y/M(t)]^L(t) - 1
        #   Zind =  -----------------
        #               S(t)L(t)
    def __init__(self, L,M,S, y):
        self.L = L # co
        self.M = M # median
        self.S = S # standard deviation
        self.y = y # age or length or weight or BMI
    
    def _zind(self, y):
        return (((y/self.M)**self.L) - 1) / (self.S * self.L)
    
    def sdx(self, X):
        ## SDpos3 = M * (1 + L * S * 3)^(1/L)
        ## SDposX = M * (1 + L * S * X)^(1/L)
        return self.M * (1 + self.L * self.S * X)**(1/self.L)
    
    def sdpos23(self):
        return self.sdx(3) - self.sdx(2)
    
    def sdneg23(self):
        return self.sdx(-2) - self.sdx(-3)
    
    def zscore(self):
        """calculate zscore for given measurement"""
        #            _
                #           |
                #           |       Zind            if |Zind| <= 3
                #           |
                #           |
                #           |       y - SD3pos
                #   Zind* = | 3 + ( ----------- )   if Zind > 3
                #           |         SD23pos
                #           |
                #           |
                #           |
                #           |        y - SD3neg
                #           | -3 + ( ----------- )  if Zind < -3
                #           |          SD23neg
                #           |
                #           |_
        res =  self._zind(self.y)
        if res >= -3 and res <=3: #≥-3 and ≤3
            return res
        elif res > 3:
            return 3 + (self.y - self.sdx(3)) / self.sdpos23()
        elif res < -3:
            return -3 + (self.y - self.sdx(-3)) / self.sdneg23()
        else:
            raise ValueError("Zind not in range")

    def z_score_to_percentile(self):
        """Calcaulte percentile from z_score"""
        return round(norm.cdf(round(self.zscore(), 2)) * 100, 2)

    def __repr__(self):
        return f"Zind(L={self.L}, M={self.M}, S={self.S}, y={self.y})"
    
    

In [335]:
def kg_to_lb(kg: float) -> float:
    return kg * 2.20462

def lb_to_kg(lb: float) -> float:
    return lb / 2.20462

def lb_and_oz_to_kg(lb: float, oz: float) -> float:
    return lb_to_kg(lb) + lb_to_kg(oz / 16)


In [336]:

@dataclass
class Child():
    """Child class with name and date of birth attributes and a method to calculate age in months
        Args:
            name (str): Name of the child
            gender (Literal['M','F'])
            dob (str): Date of birth in the format 'YYYY-MM-DD'
    """
    name: str
    gender: Literal['M', 'F', 'girl', 'boy']
    dob: str

    def __post_init__(self):
        self.age = self.get_age()
        self.months = self.get_month()
        self.dob = self.dob_validation()
        self.gender = self.get_gender()

    def dob_validation(self):
        """Date of birth cannot be in the future."""
        today = datetime.today()
        dob = datetime.strptime(self.dob, '%Y-%m-%d')
        if dob>=today:
            raise ValueError('Date of birth cannot be in the future')
        else:
            return dob
    
    def get_age(self):
        """Number of years since birth"""
        today = datetime.today()
        dob = datetime.strptime(self.dob, '%Y-%m-%d')
        age = today.year - dob.year - ((today.month, today.day) < (dob.month, dob.day))
        return age
    
    def get_months(self):    
        """Number of months since birth"""
        today = datetime.today()
        dob = datetime.strptime(self.dob, '%Y-%m-%d')
        age = today.month - dob.month + 12 * (today.year - dob.year)
        return age
    
    def get_gender(self):
        gen = {
            'm': 'boys', 
            'f': 'girls', 
            'girl': 'girls',
            'boy': 'boys'
        }
        return gen[self.gender.lower()]
    

@dataclass
class Measurement:
    """Measurement class with date and value attributes.
    
    Args:
        child (Child): Child object
        date (str): Date of the measurement in the format 'YYYY-MM-DD'
        weight (float): Weight in kg
        height (float): Height in cm
        hc (float): Head circumference in cm
        
    """
    child: Child
    date: str
    weight: Optional[float] = None
    height: Optional[float] = None
    hc: Optional[float] = None
    
    def __post_init__(self):
        self.bmi = self.get_bmi()
        self.date = self.date_validation()
        
    def get_bmi(self):
        if self.weight and self.height:
            return self.weight / (self.height ** 2)
        
    def date_validation(self):
        """Measure date cannot be in the future."""
        today = datetime.today()
        date = datetime.strptime(self.date, '%Y-%m-%d')
        print(today, date, date>today)
        if date>=today:
            raise ValueError('Date cannot be in the future')
        else:
            return date
        
    

    def __str__(self):
        return f'{self.child.name} was {self.weight} kg, {self.height} cm, and {self.hc} cm on {self.date}'
        
    
    
child = Child('John', "g", '2023-12-03')
print(child)
m = Measurement(child, '2024-03-09', 10, 70, 45)
m

Child(name='John', gender='G', dob=datetime.datetime(2023, 12, 3, 0, 0))
2024-03-11 23:19:51.006902 2024-03-09 00:00:00 False


Measurement(child=Child(name='John', gender='G', dob=datetime.datetime(2023, 12, 3, 0, 0)), date=datetime.datetime(2024, 3, 9, 0, 0), weight=10, height=70, hc=45)

In [337]:

gender = "girls" # boys
dob = datetime(2023, 12, 3)

# reference https://www.who.int/tools/child-growth-standards/standards/weight-for-age


# https://cdn.who.int/media/docs/default-source/child-growth/child-growth-standards/indicators/weight-for-age/tab_wfa_boys_p_0_5.xlsx?sfvrsn=a0b3ed5_7
# https://cdn.who.int/media/docs/default-source/child-growth/child-growth-standards/indicators/weight-for-age/tab_wfa_girls_p_0_5.xlsx?sfvrsn=666fe445_7
databases = {
    "weight": f"https://cdn.who.int/media/docs/default-source/child-growth/child-growth-standards/indicators/weight-for-age/tab_wfa_{gender}_p_0_5.xlsx",
    "height": f"https://cdn.who.int/media/docs/default-source/child-growth/child-growth-standards/indicators/length-height-for-age/tab_lhfa_{gender}_p_0_2.xlsx",
    "bmi": f"https://cdn.who.int/media/docs/default-source/child-growth/child-growth-standards/indicators/body-mass-index-for-age/tab_bmi_{gender}_p_0_2.xlsx",
    "hc": f"https://cdn.who.int/media/docs/default-source/child-growth/child-growth-standards/indicators/head-circumference-for-age/tab_hcfa_{gender}_p_0_5.xlsx"
}


In [338]:
def rename_columns(col):
    r = re.match(r'(\d+)\w+', col)
    if r:
        return int(r.group(1))
    else:
        return col

# def parse_databases(databases: Dict[str, str]) -> Dict[str, pd.DataFrame]:
#     """Parse databases and return a dictionary of dataframes"""
#     dfs = {}
#     for key, url in databases.items():
#         df = pd.read_excel(url, header=0)
#         df.columns = [rename_columns(col) for col in df.columns ]
#         dfs[key] = df
#     return dfs


def parse_table(url: str) -> pd.DataFrame:
    """Parse a table from a given url"""
    df = (
        pd
        .read_html(url, header=0)[1]
        .rename(columns={"2nd (2.3rd)": "2nd", '98th (97.7th)': '98th'})
    )
    df.columns = [rename_columns(col) for col in df.columns ]
    return df
# # url = 'https://www.cdc.gov/growthcharts/who/boys_weight_head_circumference.htm'
url  = 'https://www.cdc.gov/growthcharts/who/girls_length_weight.htm'
df = parse_table(url)   
df.head()
# df.head() 


,Month,L,M,S,2,5,10,25,50,75,90,95,98
0,0,0.3809,3.2322,0.14171,2.394672,2.532145,2.677725,2.932331,3.2322,3.550350,3.852667,4.040959,4.230430
1,1,0.1714,4.1873,0.13724,3.161067,3.326209,3.502477,3.814261,4.1873,4.590075,4.979539,5.225436,5.475454
2,2,0.0962,5.1282,0.13000,3.941053,4.131720,4.335355,4.695944,5.1282,5.596104,6.049862,6.337067,6.629679
3,3,0.0402,5.8458,0.12619,4.536040,4.745935,4.970282,5.368044,5.8458,6.364222,6.868317,7.188096,7.514480
4,4,-0.0050,6.4237,0.12402,5.013368,5.238858,5.480078,5.908320,6.4237,6.984281,7.530756,7.878150,8.233311


In [339]:
nums_col = [x for x in df.columns if isinstance(x, int)]
print(nums_col)

[2, 5, 10, 25, 50, 75, 90, 95, 98]


In [340]:
# df.plot(x='Length', y=[2, 50, 98], kind='line')


In [341]:
child1 = Child('John', 'M', '2012-12-03')
child1.age

m1 = Measurement(child1, '2024-03-09', 10, 70, 45)
m1.bmi = 30
print(m1.bmi)

z = Zind(-1.7862, 16.9392, 0.11070, m1.bmi)
print(z.zscore())
# Child 1: 11 year-old boy with BMI=30
# L=-1.7862; M=16.9392; S=0.11070; 
# answer: 3.35


z = Zind(-1.3529, 20.4951, 0.12579, 14)
print(z.zscore())
# Child 2: 16 year-old boy with BMI=14.
# L=-1.3529; M=20.4951; S=0.12579; 
# answer: -3.80


z = Zind(-1.6318, 16.0490, 0.10038, 19)
print(z.zscore())   
# Child 3: 9 year-old boy with BMI=19
# L=-1.6318; M=16.0490; S=0.10038; 
# answer: 1.47



2024-03-11 23:19:52.081479 2024-03-09 00:00:00 False
30
3.35390255606726
-3.794790928768083
1.4698319520484722


In [342]:
niki = Child('Niki', 'F', '2023-12-03')
print(niki.age, niki.months)
m2 = Measurement(niki, '2024-03-09', 5.22)
L=0.0402
M=5.8458
S=0.12619
z = Zind(L,M,S, m2.weight)
print(z.zscore())   



0 3
2024-03-11 23:19:52.166296 2024-03-09 00:00:00 False
-0.8952275022630395


In [343]:
# 3	0.0402	5.8458	0.12619	3.9	4.4	4.6	4.7	5
# 4	-0.005	6.4237	0.12402	4.4	4.8	5.1	5.2	5.5

In [344]:
z = Zind(0.0402, 5.8458, 0.12619, 5)
print(z.zscore(), z.z_score_to_percentile())

-1.2346114815021876 10.848756693001672


In [345]:
z = Zind(-0.005, 6.4237, 0.12402, 5.5)
print(z.zscore(), z.z_score_to_percentile())

-1.252269412006037 10.52358553671785


In [346]:
# df = pd.read_html('https://www.cdc.gov/growthcharts/who/girls_length_weight.htm', header=0)[1]
# df.head()
url = 'https://www.cdc.gov/growthcharts/who/girls_length_weight.htm'
df= parse_table(url) 
df



,Month,L,M,S,2,5,10,25,50,75,90,95,98
0,0,0.3809,3.2322,0.14171,2.394672,2.532145,2.677725,2.932331,3.2322,3.550350,3.852667,4.040959,4.230430
1,1,0.1714,4.1873,0.13724,3.161067,3.326209,3.502477,3.814261,4.1873,4.590075,4.979539,5.225436,5.475454
2,2,0.0962,5.1282,0.13000,3.941053,4.131720,4.335355,4.695944,5.1282,5.596104,6.049862,6.337067,6.629679
3,3,0.0402,5.8458,0.12619,4.536040,4.745935,4.970282,5.368044,5.8458,6.364222,6.868317,7.188096,7.514480
4,4,-0.0050,6.4237,0.12402,5.013368,5.238858,5.480078,5.908320,6.4237,6.984281,7.530756,7.878150,8.233311
5,5,-0.0430,6.8985,0.12274,5.403844,5.642267,5.897544,6.351329,6.8985,7.495018,8.077933,8.449225,8.829415
6,6,-0.0756,7.2970,0.12204,5.729383,5.978880,6.246243,6.722120,7.2970,7.925102,8.540297,8.932890,9.335491
7,7,-0.1039,7.6422,0.12178,6.008387,6.267836,6.546104,7.042017,7.6422,8.299352,8.944440,9.356859,9.780399
8,8,-0.1288,7.9487,0.12181,6.253445,6.522061,6.810403,7.324907,7.9487,8.633118,9.306424,9.737639,10.181094
9,9,-0.1507,8.2254,0.12199,6.472906,6.750018,7.047717,7.579535,8.2254,8.935413,9.635310,10.084290,10.546619


In [398]:
LMS_cols = ['L', 'M', 'S']

nikki = Child('Nikki', 'F', '2023-12-03')


subject = pd.DataFrame({
    'date': ['2024-03-08', '2024-03-06', '2024-02-24', '2024-02-12', '2024-02-05'],
    'weight_lb': [11, 11, 11, 10, 10],
    'weight_oz': [8.2, 7, 0, 4.4, 2],
    'huk_percent': [16, 16, 16, 16, 17]

})

def get_LSM(df, month):
    values = df[df['Month'] == month][LMS_cols].values[0]
    print(values)
    return values


subject['weight_kg'] = lb_and_oz_to_kg(subject['weight_lb'], subject['weight_oz'])
subject['months'] = subject['date'].apply(lambda x: (datetime.strptime(x, '%Y-%m-%d') - niki.dob).days / 30)
subject['percentile'] = subject.apply(lambda x: Zind(*get_LSM(df, round(x['months'],0)), x['weight_kg']).z_score_to_percentile(), axis=1)
subject

[0.0402  5.8458  0.12619]
[0.0402  5.8458  0.12619]
[0.0402  5.8458  0.12619]
[0.0962 5.1282 0.13  ]
[0.0962 5.1282 0.13  ]


,date,weight_lb,weight_oz,huk_percent,weight_kg,months,percentile
0,2024-03-08,11,8.2,16,5.221988,3.200000,18.67
1,2024-03-06,11,7.0,16,5.187969,3.133333,17.36
2,2024-02-24,11,0.0,16,4.989522,2.766667,10.56
3,2024-02-12,10,4.4,16,4.660667,2.366667,23.27
4,2024-02-05,10,2.0,17,4.592628,2.133333,20.05


In [348]:
subject


,date,weight_lb,weight_oz,huk_percent,weight_kg,months,percentile
0,2024-03-08,11,8.2,16,5.221988,3.200000,18.613666
1,2024-03-06,11,7.0,16,5.187969,3.133333,17.264189
2,2024-02-24,11,0.0,16,4.989522,2.766667,10.544338
3,2024-02-12,10,4.4,16,4.660667,2.366667,23.208922
4,2024-02-05,10,2.0,17,4.592628,2.133333,19.933660


In [362]:
# def cleanup_weight(weight):
#     if weight is None or pd.isna(weight):
#         return np.nan
#     if 'lb' in weight:
#         # return lb_and_oz_to_kg(*map(float, weight.replace('lbs.oz','').split(".",1)))
#         return lb_to_kg(float(weight.replace('lbs.oz', '')))
#     else:
#         return float(weight.replace('kg', ''))

# nikki_df = (
#     pd
#     .read_csv('Nikki/22ef57f2-8b3a-4d7f-8e0a-f4f015100e40.csv')
#     .query('Type  == "Growth"')
#     .rename(columns={
#         'Start': 'date', 
#         'Start Condition': 'weight',

#         "Start Location": 'height',
#         'End Condition': 'hc'
#         })
#     .dropna(axis=1, how='all')
    
#     )

# nikki_df['date'] = pd.to_datetime(nikki_df['date'])
# nikki_df['weight_kg'] = nikki_df['weight'].apply(cleanup_weight)


# nikki_df

nikki_df = huckleberry_reader('Nikki/22ef57f2-8b3a-4d7f-8e0a-f4f015100e40.csv')
nikki_df['months'] = (nikki_df['date'] - niki.dob).dt.days / 30
nikki_df['percentile'] = nikki_df.apply(lambda x: Zind(*get_LSM(df, round(x['months'],0)), x['weight_kg']).z_score_to_percentile(), axis=1).round(2)
nikki_df



nan
nan
nan
nan
nan
1.87ft.in
1.88ft.in
nan
54cm
nan
nan
1.74ft.in
nan
1.75ft.in
nan
nan
nan
nan
nan
14.5in
nan
nan
13.8in
nan
nan
13.5cm
nan
nan
[0.0402  5.8458  0.12619]
[0.0402  5.8458  0.12619]
[0.0402  5.8458  0.12619]
[0.0402  5.8458  0.12619]
[0.0962 5.1282 0.13  ]
[0.0962 5.1282 0.13  ]
[0.0962 5.1282 0.13  ]
[0.0962 5.1282 0.13  ]
[0.1714  4.1873  0.13724]
[0.1714  4.1873  0.13724]
[0.1714  4.1873  0.13724]
[0.3809  3.2322  0.14171]
[0.3809  3.2322  0.14171]
[0.3809  3.2322  0.14171]


,Type,date,weight,height,hc,weight_kg,height_cm,hc_cm,months,percentile
54,Growth,2024-03-08 16:23:00,11.51lbs.oz,NaN,NaN,5.220844,<NA>,<NA>,3.200000,18.57
125,Growth,2024-03-06 01:23:00,11.44lbs.oz,NaN,NaN,5.189092,<NA>,<NA>,3.133333,17.31
414,Growth,2024-02-24 15:10:00,4.99kg,NaN,NaN,4.990000,<NA>,<NA>,2.766667,10.56
509,Growth,2024-02-21 07:01:00,10.84lbs.oz,NaN,NaN,4.916937,<NA>,<NA>,2.666667,8.59
766,Growth,2024-02-12 08:24:00,4.66kg,NaN,NaN,4.660000,<NA>,<NA>,2.366667,23.18
977,Growth,2024-02-05 15:59:00,10.13lbs.oz,1.87ft.in,14.5in,4.594887,56.9976,36.83,2.133333,20.04
1082,Growth,2024-02-02 13:54:00,9.81lbs.oz,1.88ft.in,NaN,4.449738,57.3024,<NA>,2.033333,13.91
1263,Growth,2024-01-25 16:08:00,4.34kg,NaN,NaN,4.340000,<NA>,<NA>,1.766667,10.14
1587,Growth,2024-01-11 09:47:00,8.75lbs.oz,54cm,13.8in,3.968930,54.0,35.052,1.300000,34.88
1777,Growth,2024-01-02 22:45:00,4kg,NaN,NaN,4.000000,<NA>,<NA>,1.000000,36.99


In [380]:
import numpy as np

def C_alpha(M, L, S, zscore):
  """
  This function implements the formula C(t) = M(t)[1 + L(t)S(t)Z_alpha]**(1/L(t))

  Args:
      t: Time variable.
      M: Function of time t.
      L: Function of time t.
      S: Function of time t.
      Z_alpha: Value between -3 and 3.

  Returns:
      The value of C(t).
  """

  # Ensure Z_alpha is within the constraint
  if zscore < -3 or zscore > 3:
    raise ValueError("Z-score must be between -3 and 3")

  # Implement the formula
  return M * (1 + L * S * zscore)**(1/L)


L,M,S = [0.3809, 3.2322,  0.14171]
z = Zind(L,M,S, 3.170608)
print(z.zscore(), z.z_score_to_percentile())

print(C_alpha(M, L, S, z.zscore()))



-0.13527151540037136 44.6198602734104
3.1706080000000005


In [368]:
import seaborn as sns
# sns.color_palette("Spectral", as_cmap=True)
colors = sns.color_palette("icefire",len(nums_col)).as_hex()

print(colors)
percentiles_to_color = dict(zip(nums_col, colors))


['#75b8ce', '#3885d0', '#4a4fa5', '#302e4a', '#1f1e1e', '#4a252e', '#932e44', '#d34936', '#f18f51']


In [400]:
from decimal import Decimal as D
D(3.170608).quantize(D('0.01'))

Decimal('3.17')

In [372]:

def plot_growth(df,col):
    return Scatter(
        x=df['Month'],
        y=df[col],
        mode='lines',
        name=f'{col}th',
        line=dict(
            shape = 'linear',
            color = percentiles_to_color.get(col, 'rgb(205, 12, 24)'),
            width= 1,
            dash = 'solid'
        ) if col in percentiles_to_color else {}
    )


# line_sytle = {
#     2: dict(shape = 'linear', color = 'rgb(205, 12, 24)', width= 1, dash = 'dash'),
#     10: dict(shape = 'linear', color = 'rgb(205, 12, 24)', width= 1, dash = 'dot'),

# }

plots = []
for col in nums_col:
    # trace = Scatter(
    #     x=df['Month'],
    #     y=df[col],
    #     mode='lines',
    #     name=f'{col}th',
    #     line = line_sytle.get(col, {}),
    #     marker=dict(
    #         size=.5,
    #         # color='red'
    #         # color='rgba(152, 0, 0, .8)',
    #         # line=dict(
    #         #     width=2,
    #         #     color='rgb(0, 0, 0)'
    #         # )
    #     )
    # )
    trace = plot_growth(df, col)
    plots.append(trace)


## add hover info for each subject
trace = Scatter(
    x=nikki_df['months'],
    y=nikki_df['weight_kg'],
    mode='markers',
    name='Subject',
    text=nikki_df['percentile'].astype(str) + '%',  
    # hoverinfosrc=subjects['percentile'],
    # hoverinfo='name',
    # hoveron=subjects,
    marker=dict(
        size=5,
        color='red',
        # color='rgba(152, 0, 0, .8)',
        # line=dict(
        #     width=2,
        #     color='rgb(0, 0, 0)'
        # )
    )
)

plots.append(trace)
iplot(plots)

In [373]:
# C_alpha(1, 16.9392, -1.7862, 0.11070, 3.35)


ValueError: Z-score must be between -3 and 3

In [ ]:
lb_and_oz_to_kg(10,4.4)
lb_to_kg(10.84)

4.916947138282334

In [ ]:
lb_and_oz_to_kg(10,13.5)


4.918648111692718

In [ ]:
'10.13.5'.split(".",1)  

['10', '13.5']

In [ ]:
from pathlib import Path
from collections import defaultdict


bmi.girls.0_2
bmi.boys.0_2
lhfa.boys.0_2
bmi.girls.2_5
lhfa.boys.2_5
bmi.boys.2_5
hcfa.girls.0_5
wfa.boys.0_5
lhfa.girls.2_5
wfa.girls.0_5
lhfa.girls.0_2
hcfa.boys.0_5
Memory usage of dfs: 192 bytes


In [ ]:
dfs['girls']['wfa'][0, 5].info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 19 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Month   61 non-null     int64  
 1   L       61 non-null     float64
 2   M       61 non-null     float64
 3   S       61 non-null     float64
 4   P01     61 non-null     float64
 5   P1      61 non-null     float64
 6   P3      61 non-null     float64
 7   P5      61 non-null     float64
 8   P10     61 non-null     float64
 9   P15     61 non-null     float64
 10  P25     61 non-null     float64
 11  P50     61 non-null     float64
 12  P75     61 non-null     float64
 13  P85     61 non-null     float64
 14  P90     61 non-null     float64
 15  P95     61 non-null     float64
 16  P97     61 non-null     float64
 17  P99     61 non-null     float64
 18  P999    61 non-null     float64
dtypes: float64(18), int64(1)
memory usage: 9.2 KB


In [404]:
# for _, dfx in df[['L','M','S',25]].iterrows():
#     # print(dfx)  
    
#     LMS = dfx[['L','M','S']].values 
#     # print(LMS)
#     print(Zind(*LMS, dfx[25]).z_score_to_percentile().round(2))

df.head().to_csv('test.csv', index=False)
    

In [417]:
import pandas as pd
from scipy.stats import norm

def interpolate_lms(age, data):
    """
    Interpolate L, M, and S values for a given age using linear interpolation.
    
    Args:
        age (float): The age in months.
        data (pandas.DataFrame): The data frame containing L, M, and S values.
        
    Returns:
        tuple: A tuple containing the interpolated L, M, and S values.
    """
    age_lower = data.index[data.index <= age].max()
    age_upper = data.index[data.index >= age].min()
    
    age_diff = age_upper - age_lower
    age_frac = (age - age_lower) / age_diff
    
    L = data.loc[age_lower, 'L'] + age_frac * (data.loc[age_upper, 'L'] - data.loc[age_lower, 'L'])
    M = data.loc[age_lower, 'M'] + age_frac * (data.loc[age_upper, 'M'] - data.loc[age_lower, 'M'])
    S = data.loc[age_lower, 'S'] + age_frac * (data.loc[age_upper, 'S'] - data.loc[age_lower, 'S'])
    
    return L, M, S

def weight_for_age_z_score(weight, L, M, S):
    """
    Calculate the z-score for weight-for-age using the LMS method.
    
    Args:
        weight (float): The child's weight in kg.
        L (float): The Box-Cox power parameter.
        M (float): The median weight.
        S (float): The coefficient of variation.
        
    Returns:
        float: The z-score for weight-for-age.
    """
    z_ind = ((weight / M) ** L - 1) / (L * S)
    return z_ind

def weight_for_age_percentile(z_score):
    """
    Calculate the growth percentile from the z-score using the standard normal distribution.
    
    Args:
        z_score (float): The z-score for weight-for-age.
        
    Returns:
        float: The growth percentile.
    """
    return 100 * norm.cdf(z_score)

# Load the data from the CSV file
data = pd.read_csv('test.csv', index_col='Month')

# Example usage
age = 2  # Age in months
weight = 3.35  # Weight in kg

# Interpolate L, M, and S values for the given age
L, M, S = interpolate_lms(age, data)

# Calculate z-score and percentile
# z_score = weight_for_age_z_score(weight, L, M, S)
z_score = Zind(L, M, S, weight).zscore()
percentile = weight_for_age_percentile(z_score)

print(f"Age: {age} months, Weight: {weight} kg")
print(f"Z-score for weight-for-age: {z_score:.2f}")
print(f"Growth percentile: {percentile:.2f}%")

/var/folders/wl/gxtj7gr176jczz5qf034r6gc0000gn/T/ipykernel_31219/4097568179.py:19: RuntimeWarning:

invalid value encountered in long_scalars



ValueError: Zind not in range